# Let's build a Todo app! 🛠️📚

## Setup 📄

1. Open the terminal using the top menu
2. Set up a python virtual environment in the terminal with this command `python3 -m venv .venv`
3. Activate the virtual environment with this command `source .venv/bin/activate`

4. Run the `Hello World` code in the next cell and set your kernal for Jupyter notebooks, selected *"Select Kernal"* at the top to **.venv** and click "Install" on the pop up.


In [1]:
print("Hello, World!")

Hello, World!


## Making the app work

1. Run the Flask app using this command `python3 -m flask run --port 5000 --debug`
2. Complete `YOUR_SQL_QUERY_HERE`s in the code inside `app/service_todo/sql/*.py` files with your SQL queries. 

Find the database schema in `app/schema.sql` for reference

## Playground 🤾🛝

Change `query` to any other SQL queries you want to test

### SELECT queries

In [6]:
import sqlite3
import pandas as pd

# Connect to the SQLite database file
conn = sqlite3.connect('instance/todo.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Example: List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Run a custom SQL query
query = """
SELECT * FROM todo 
ORDER BY created_at DESC LIMIT 10;
"""
# Using pandas to directly load and display query results
df = pd.read_sql_query(query, conn)
print(df.head())

# Close the connection when finished
conn.close()


Tables: [('todo',), ('subtodo',)]
      id     title     desc  completed           created_at  \
0  my_id  my_title  my_desc          1  2025-04-10 21:58:24   

            updated_at due_date  
0  2025-04-10 21:58:24     None  


### INSERT/UPDATE/DELETE queries

In [5]:
import sqlite3

# Connect to the SQLite database file
conn = sqlite3.connect('instance/todo.db', timeout=10)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Run a custom SQL query that modifies the database
# WARNING: The query will modify the database
query = """
INSERT INTO todo (id, title, desc, completed)
VALUES ('my_id', 'my_title', 'my_desc', 1)
"""

# Execute the query
cursor.execute(query)
# Commit the changes to the database
conn.commit()

# Close the connection when finished
conn.close()



## Let's add a due date to your todos! ✅

1. Go to the schema.sql file and add `due_date` to the todo and subtodo table 
   with type `TIMESTAMP` and default value `NULL`
2. Go to the type_defs.py file, add `from datetime import datetime` to the imports and 
   add due_date to the `Todo` and `SubTodo` classes with type `Optional[datetime]`

   ```python
   from datetime import datetime

   class Todo(TypedDict):
     # ...
     due_date: Optional[datetime]

   class SubTodo(TypedDict):
     # ...
     due_date: Optional[datetime]
   ```

### Parse due_date to Python `datetime` 🔀

3. In todo_ins.py and todo_upd.py, add `from datetime import datetime` to the imports and
   paste the following code after the `data: _ReqBody = request.get_json()` line
   to parse the `due_date` to a `datetime` object:

   ```python
   from datetime import datetime
   # ...

   data: _ReqBody = request.get_json()
   data['todo']['due_date'] = datetime.fromisoformat(data['todo']['due_date'])
   ```
4. Do the same thing in subtodo_ins.py and subtodo_upd.py, but for both `todo` and `subtodo`:

   ```python
   from datetime import datetime
   # ...

   data: _ReqBody = request.get_json()
   data['todo']['due_date'] = datetime.fromisoformat(data['todo']['due_date'])
   data['subtodo']['due_date'] = datetime.fromisoformat(data['subtodo']['due_date'])
   ```

### Update your SQL queries 💬

5. Modify sql_todo_get.py to include `due_date` when parsing the row to `Todo` in the response
   `due_date` is automatically a `datetime` object (or `None`), so you don't need to do any conversion
6. Modify sql_todos_get.py to include `due_date` when parsing the rows to `List[Todo]` in the response
7. Modify sql_todo_ins.py to include `due_date` in the SQL query and the parameters
8. Modify sql_todo_upd.py to include `due_date` in the SQL query and the parameters
9. Modify sql_subtodo_get.py to include `due_date` when parsing the row to `SubTodo` in the response
10. Modify sql_subtodos_get.py to include `due_date` when parsing the rows to `List[SubTodo]` in the response
11. Modify sql_subtodo_ins.py to include `due_date` in the SQL query and the parameters
12. Modify sql_subtodo_upd.py to include `due_date` in the SQL query and the parameters

### Update your UI 🖼️

13.  Go to the static/index.html file, search for the TODO comment and add the following code:

     ```html
     <!-- TODO: Add new todo due date input below -->
     <label>Due Date
       <input type="datetime-local" id="new-todo-due-date">
     </label>
     ```
14.  Go to the static/index.js file, search for the TODO comments and add the following code:
    
     ```html
     <!-- TODO: Add edit subtodo due date input below -->
     <label>Due Date
       <input type="datetime-local" class="edit-subtodo-due-date" 
         value="${subtodo.due_date ? formatDateToDatetimeLocal(new Date(subtodo.due_date)) : ''}"
       >
     </label>
 
     <!-- TODO: Add edit todo due date input below -->
     <label>Due Date
       <input type="datetime-local" class="edit-todo-due-date"
         value="${todo.due_date ? formatDateToDatetimeLocal(new Date(todo.due_date)) : ''}"
       >
     </label>
 
     <!-- TODO: Add new subtodo due date input below -->
     <label>Due Date
       <input type="datetime-local" class="new-subtodo-due-date">
     </label>
     ```

     ```js
     // TODO: Add new todo due date field below
     const todoDueDateValue = document.getElementById('new-todo-due-date').value;

     // TODO: Reset the new todo due date field
     document.getElementById('new-todo-due-date').value = '';

     // TODO: Add edit todo due date field below
     const todoDueDateValue = todoLI.querySelector('.edit-todo-due-date').value;

     // TODO: Add edit subtodo due date field below
     const subtodoDueDateValue = subtodoLI.querySelector('.edit-subtodo-due-date').value;

     // TODO: Add new todo and subtodo due date field below
     const todoDueDateValue = todoLI.querySelector('.edit-todo-due-date').value;
     const subtodoDueDateValue = form.querySelector('.new-subtodo-due-date').value;

     const reqBody = {
       todo: {
         // ...,
         // TODO: Add the todo due date to the request body (there are 5 places)
         due_date: todoDueDateValue ? formatDateToSQLite(new Date(todoDueDateValue)) : null
       },
       subtodo: {
         // ...,
         // TODO: Add the subtodo due date to the request body (there are 2 places)
         due_date: subtodoDueDateValue ? formatDateToSQLite(new Date(subtodoDueDateValue)) : null
       }
     }
     ```

### Test your changes 🎉

15. Delete the database file `instance/todo.db` and run the command 
    `python3 -m flask run` to restart the server and create a new database with the new schema
16. Open the web app in your browser and test your changes!
